## work in progress of heasoft analysis for BAT data for FRBS yada yada

In [1]:
# imports
import numpy as np
import matplotlib as pyplot
from astropy.io import fits


import os, sys, time, shutil
import astropy.io.fits as pyfits
import numpy as np
import re
#from swiftmonitor.utils import execute_cmd, region

import subprocess # for running shell commands

In [8]:
results = '../results' # maybe I'll save stuff in the corresponding swift id dir? that seems reasonable 
file_ext = '.pdf' # for the outputted plots
data_dir = '../data' # will loop through every dir for the bat data

# args
clobber = True # if true remakes/overwrites files
time_res = 1 # seconds? 
energy_bins = 14-175
source_id = '00010374156'
evt_file = 'sw00010374156bevshsl_uf.evt.gz'


events = '/bat/event'
event_dir = data_dir + '/' + source_id + events
hk_dir = data_dir + '/' +source_id + '/bat/hk'
results_dir = results + '/' + source_id

In [ ]:
# to do
# need to make a wrapper function for all of this
# need to add a subproccess thing to log file
# maybe add a progress bar 
# need to make a forloop
# make better SNR search methods
# figure out rate of false concidence 
# organise a results dir 
# come up with a pipeline name?

# pipeline steps
    
    # make mask
    # make LC using heafsoft
        # take args for energy bands to look at, and time
        # plot LC
        # search for signals maybe
        
    # make frb_dpi
    # make back_dpi 
        # arguments for bck time, and DPI need energy bands and time 
        
    # make sky image
    # run heafsoft source finder, snr min for that is 3.5
        # record results to file
        # compare ra/dec of any potential signals to chime broad band localizatons
        
    # make my own LC around FRB ra/dec to establish limits
        # args for LC


In [1]:
#event_dir

In [45]:
event_dir, hk_dir, results_dir = make_dir(source_ids,data_dir,results)

In [52]:
#source_ids #= get_swift_ids_data(data_dir)

In [49]:
def get_swift_ids_data(data_dir):
    '''make list of swift IDS from the data dir'''
    output = subprocess.check_output(["ls",],universal_newlines=True, cwd= data_dir)
    source_ids = output.split() # list of strings the source IDs 
    
    # pop off these dirs if they exist in the list
    source_ids.remove('GUANO')
    source_ids.remove('dump')
    source_ids.remove('inventory')
    source_ids.remove('-')
    source_ids.remove('GUANO')
    source_ids.remove('triggers.tsv')
    source_ids.remove('skycoords.dat')
    return source_ids

#bat_event_data = subprocess.check_output(["ls",],universal_newlines=True, cwd= event_dir)
#bat_event_files = bat_event_data.split()

In [ ]:
# loop through data, and execute lc commands from heasoft commands

subprocess.run(['pwd'], cwd = data_dir)
# obtain list of all files in ../data
output = subprocess.check_output(["ls",],universal_newlines=True, cwd= data_dir)
source_ids = output.split() # list of strings the source IDs 

# want to sort to only do the strings that have localizations

events = '/bat/event'
#bat_event_dir = data_dir + source_id[i] + events

# making lc fits 

#source_id[0] = '03110142001'

event_dir = data_dir + '/' + source_id[0] + events
hk_dir = data_dir + '/' +source_id[0] + '/bat/hk'
results_dir = results + '/' + source_id[0]
#results_dir2 = 

bat_event_data = subprocess.check_output(["ls",],universal_newlines=True, cwd= event_dir)
bat_event_files = bat_event_data.split()
    
for j in bat_event_files:
    if j[-6:] != 'evt.gz': # check extension to be sure we only look at '.evt.gz' file types
        # maybe do bevshsp_uf.evt instead? 
         bat_event_files.remove(j) 
            # this doesnt work as intended! need to update

    infile = j
    out_lc_fitsout= str(j) + 'out.lc' # make a results dir? yes
   # frb_dpi = str(j) + '.chime.frb.dpi'
    bck_dpi = str(j) + '.bck.chime.frb.dpi'
    
    
    # pipeline steps
    
    # make mask
    # make LC using heafsoft
        # take args for energy bands to look at, and time
        # plot LC
        # search for signals maybe
        
    # make frb_dpi
    # make back_dpi 
        # arguments for bck time, and DPI need energy bands and time 
        
    # make sky image
    # run heafsoft source finder, snr min for that is 3.5
        # record results to file
        # compare ra/dec of any potential signals to chime broad band localizatons
        
    # make my own LC around FRB ra/dec to establish limits
        # args for LC
        
        
    # make entry in results dir based on swift id    
    subprocess.run(['mkdir ' + str(source_id[0])], cwd= results, shell=True) # change 0 to i when making wrapper!!
    
    # make DPI 
    subprocess.run(['batbinevt ' + str(j) + ' weighted=no outunits=counts outtype=dpi energy=-' 
                   +' clobber=' + str(clobber)
                     +' outfile= frb.dpi'] , cwd = event_dir, shell=True)
    
    # move frb.dpi and (copy) det mask, event file. attitude file into results dir 
    subprocess.run(['mv ' +str(event_dir) + '/frb.dpi ' 
                     + str(results_dir) + '/frb.dpi'  ], 
                    shell=True, cwd = '../data')
    
    subprocess.run(['cp ' +str(hk_dir) + '/sw' + str(source_id[0]) + 'bdecb.hk.gz ' 
                   + str(results_dir) + '/sw' + str(source_id[0]) + 'bdecb.hk.gz'  ], 
                   shell=True, cwd = '../data')
    
    subprocess.run(['cp data/' +str(source_id[0]) + '/auxil/sw' + str(source_id[0]) + 'sat.fits.gz ' 
                   + 'data/' + str(results_dir) + '/sw' + str(source_id[0]) + 'sat.fits.gz' ], 
                   shell=True, cwd = '..')
    
    subprocess.run(['cp data/' +str(source_id[0]) + '/bat/event/' + str(j) + ' ' 
                   + 'data/' + str(results_dir) + '/' + str(j) ], 
                   shell=True, cwd = '..')
    
    # make mask
    subprocess.run(['bathotpix detmask=sw' + str(source_id[0]) 
                    + 'bdecb.hk.gz outfile = frb.mask clobber =' + str(clobber) ],
                   shell=True, cwd=results_dir)
    
    # make lc with heasoft 
    # should I set energy bins here and time res?
    subprocess.run(['batbinevt detmask=frb.mask ' + str(j) 
                + ' timedel = ' + str(time_res)  + ' weighted=no outtype=lc energybins=- outfile =frb.heasoft.lc clobber =' 
                + str(clobber) ],
                   shell=True, cwd=results_dir)
    
   # make frb2_dpi
   # this one should contain our target source
   # need to add target_time and time size stuff
   # I am unsure if the timedel is correct here
    subprocess.run(['batbinevt '  + str(j) 
                + ' outfile =frb2.dpi energybins=' +str(energy_bins) 
                + ' clobber= ' + str(clobber) + ' outtype=dpi detmask=frb.mask timedel = 0' 
                + ' tstart=639072334.000000 tstop=639072896.800600' ],
               shell=True, cwd=results_dir)
    
    # make bkg.dpi
    # this one uses a off event time size to make a background dpi
    subprocess.run(['batbinevt '  + str(j) 
                + ' outfile =bkg.dpi energybins=' +str(energy_bins)  
                + ' clobber= ' + str(clobber) + ' outtype=dpi detmask=frb.mask timedel = 0' 
                + ' tstart=639072334.000000 tstop=639072896.800600' ],
               shell=True, cwd=results_dir)
    
    # make sky image
    subprocess.run(['batfftimage frb2.dpi attitude = sw' + str(source_id[0]) + 'sat.fits.gz detmask=frb.mask'  
                   + ' outfile = frb.sky bkgfile=bkg.dpi clobber =' + str(clobber)], 
                   shell=True, cwd = results_dir)
    
    # run heasoft source finder
    subprocess.run(['batcelldetect frb.sky outfile=cat.fits'], shell=True, cwd = results_dir)
    
    # need to make
    # log file, plotting funciton focus
    # progress bar
    # script focus
    # args for search focus
    # extract lc stuff focus
    # Time trigger stuff focus

        
        

In [ ]:
clobber = True

In [ ]:
source_id[0]

In [ ]:
j

In [12]:
run_heasoft(source_id, evt_file, results, event_dir, hk_dir, 
                results_dir, clobber=clobber, energy_bins=14-175, time_res=1)

mkdir: cannot create directory ‘00010374156’: File exists


******************************************
         batbinevt v1.48
------------------------------------------
    Input Events: sw00010374156bevshsl_uf.evt.gz
     Output File: frb.dpi  (DPI)
   Detector Mask: NONE
      Time Range: 639072896.800600 to 639072956.140000  (requested)
      Time Range: 639072896.800600 to 639072956.140000  (actual)
Apply Weighting?: NO      Energy Column: ENERGY
     Energy Bins: -
    Output Units: COUNTS
  Binning Method: u = UNIFORM
   Time Bin Size: 0.000000 (s)
------------------------------------------
  DPIs written to frb.dpi
  EBOUNDS written to frb.dpi
    Number of Rows Processed: 413571
    Number Accepted/Rejected: 413571/0
  Time Bins:          1      Energy Bins:    1
------------------------------------------
******************************************
         bathotpix v1.10
------------------------------------------
     Input Image: frb.dpi
          Number: -1
------------------------------------------
   Image maximum value: 154 
   

******************************************
         batfftimage v1.20
------------------------------------------
     Input Image: frb2.dpi
 Background File: bkg.dpi
   Attitude File: sw00010374156sat.fits.gz
     TelDef File: CALDB
  Aperture Image: CALDB:FLUX
    Output Image: frb.sky
   Detector Mask: frb.mask
    Oversampling:  2 X    2 Y
  Partial Coding: NO    Threshold: 0.010000
Rebalance Images: YES
     Corrections: autocollim=YES handedness=left  pcode=YES flatfield=YES keepbits=7 
------------------------------------------
  Sky image 1 written to frb.sky
    Sky images computed: 1
------------------------------------------
******************************************
#        batcelldetect v1.85
------------------------------------------
     Input Image: frb.sky
  Output Catalog: cat.fits  
   Input Catalog: NONE
Part. Coding Map: NONE
    Back. Window: CIRCLE    Radius: 30
   Source Window: CIRCLE    Radius: 6
   SNR Threshold: 3.500000
 Number of Iter.: 2
Min. Num. Pixels:

In [ ]:
run_heasoft

In [11]:
run_heasoft(source_id, results, event_dir, hk_dir, 
                results_dir, clobber=clobber, energy_bins=14-175, time_res=1)

TypeError: run_heasoft() missing 1 required positional argument: 'results_dir'